In [ ]:
import os, json, datetime, sys
from azureml.core import Workspace, Environment

from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.environment import Environment

ws = Workspace.from_config()

In [ ]:
model = Model(ws, 'arima_model.pkl')  #, version=4)

In [ ]:
%%writefile score.py
import pickle
import json
import numpy
from azureml.core.model import Model
from statsmodels.tsa.arima_model import ARIMA


def init():
    global model
    import joblib

    # load the model from file into a global object
    model_path = Model.get_model_path(model_name="arima_model.pkl")
    model = joblib.load(model_path)


def run(raw_data):
    try:
        data = json.loads(raw_data)["data"]
        data = numpy.array(data)
        result=model.forecast(steps=data[0])[0]
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

In [ ]:
myenv = Environment.from_conda_specification(name="arima-env", file_path="./arima-env.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=3, 
                                               tags={'name':'arima-inference', 'framework': 'statsmodels'},
                                               description='arima inference')

service = Model.deploy(workspace=ws,
                           name='arima-inference', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig, overwrite=True)

service.wait_for_deployment(True)
print(service.state)

In [ ]:
print(service.get_logs())

In [ ]:
print(service.scoring_uri)

In [ ]:
step_size=[3]
test_sample = json.dumps({"data": step_size})
test_sample = bytes(test_sample, encoding="utf8")
print(test_sample)

In [ ]:
prediction = service.run(input_data=test_sample)
print(prediction)